In [1]:
#Author: 陳冠昇
#Student ID: 0816036
#HW ID: HW2

In [2]:
import spacy
import numpy as np
import pandas as pd

c:\Users\Terry\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
nlp = spacy.load("en_core_web_trf")

Inference = 1

if Inference:
    df = pd.read_csv("./dataset.csv", header=None, names = ["index", "Sentence", "S", "V", "O"])
    sentences = df["Sentence"]
    S = df["S"].to_list()
    S = [str(s).split(" ") for s in S]
    V = df["V"].to_list()
    V = [str(v).split(" ") for v in V]
    O = df["O"].to_list()
    O = [str(o).split(" ") for o in O]
else:
    df = pd.read_csv("./example_with_answer.csv")
    label = df["T/F"]
    sentences = df["Sentence"]
    S = df["S"].to_list()
    S = [s.split(" ") for s in S]
    V = df["V"].to_list()
    V = [v.split(" ") for v in V]
    O = df["O"].to_list()
    O = [o.split(" ") for o in O]



# print(sentences)

In [4]:

def add_valid(id, select_SVO, subj, verb, obj):
    # add ---------------------------------------------------------
    if subj != None and obj != None:
        tmp = [subj, verb, obj]
        select_SVO[id].append(tmp)
    return select_SVO

In [5]:
def find_objects(id, select_SVO, doc, subj, subj_id, verb_id, t):
    assert subj != None
    obj = None
    token = doc[verb_id]
    subtree = list(token.subtree)
    for i in range(0, len(subtree)):
        t = subtree[i]

        if token.pos_ == "AUX" and t.dep_ == "attr": # AUX as root
            subsubtree = list(t.subtree)
            # attr
            obj = t.text
            select_SVO = add_valid(id, select_SVO, subj, token.text, obj)
            # attr prep pobj
            for st in subsubtree:
                if st.dep_ == "prep" and st.head == t:
                    for sst in list(st.subtree):
                        if sst.dep_ == "pobj" and sst.head == st:
                            obj = sst.text
                            select_SVO = add_valid(id, select_SVO, subj, token.text, obj)
        elif t.dep_ == "dobj" and t.head == token:
            subsubtree = list(t.subtree)
            # dobj
            obj = t.text
            select_SVO = add_valid(id, select_SVO, subj, token.text, obj)
            # dobj prep pobj
            for st in subsubtree:
                if st.dep_ == "prep" and st.head == t:
                    for sst in list(st.subtree):
                        if sst.dep_ == "pobj" and sst.head == st:
                            obj = sst.text
                            select_SVO = add_valid(id, select_SVO, subj, token.text, obj)


        elif t.dep_ == "prep" and t.head == token:
            subsubtree = list(t.subtree)
            # prep pobj
            for st in subtree:
                if st.dep_ == "pobj" and st.head == t:
                    obj = st.text
                    select_SVO = add_valid(id, select_SVO, subj, token.text, obj)

        elif t.dep_ == "xcomp":
            subsubtree = list(t.subtree)
            # xcomp prep pobj
            for st in subsubtree:
                if st.dep_ == "prep" and st.head == t:
                    for sst in list(st.subtree):
                        if sst.dep_ == "pobj" and sst.head == st:
                            obj = sst.text
                            select_SVO = add_valid(id, select_SVO, subj, token.text, obj)


        elif t.dep_ == "advmod" and t.head == token:
            obj = t.text
            select_SVO = add_valid(id, select_SVO, subj, token.text, obj)

        
        # select_SVO = add_valid(id, select_SVO, subj, token.text, obj)

    
    return select_SVO

In [6]:
from audioop import reverse


def parse_sentence(id, select_SVO, doc, verb_id):
    subj = None
    obj = None
    token = doc[verb_id]
    subtree = list(token.subtree)

    for i, t in enumerate(subtree):
        if (t.dep_ == "nsubj" or t.dep_ == "nsubjpass") and t.head == token:
            if t.text == "who":
                if t.i-1 >= 0:
                    subj = doc[t.i-1].text # covered by later subj
                    if subj == ",":
                        subj_id = doc[t.i-1].i-1
                    else:
                        subj_id = doc[t.i-1].i

                    
                    # select_SVO = add_valid(id, select_SVO, subj, token.text, obj)
                    select_SVO = find_objects(id, select_SVO, doc, subj, subj_id, verb_id, t)
                before_who = t.i
                for j in reversed(range(t.i + 1)):
                    if doc[j].dep_ == "nsubj" or doc[j].dep_ == "pobj":
                        before_who = j
                        subj = doc[before_who].text
                        subj_id = doc[t.i-1].i
                        # select_SVO = add_valid(id, select_SVO, subj, token.text, obj)
                        select_SVO = find_objects(id, select_SVO, doc, subj, subj_id, verb_id, t)

                # subj = doc[before_who].text # head
                
            else:
                subj = t.text
                subj_id = doc[t.i-1].i
                # select_SVO = add_valid(id, select_SVO, subj, token.text, obj)
                select_SVO = find_objects(id, select_SVO, doc, subj, subj_id, verb_id, t)
            #-----------------------------------------------

    return select_SVO


In [7]:
y_pred = [0]*len(sentences)
select_SVO = [[] for i in range(len(sentences))]
s_pred = [None]*len(sentences)
v_pred = [None]*len(sentences)
o_pred = [None]*len(sentences)

for id, sentence in enumerate(sentences):
    subj = None
    verb = None
    obj = None
    doc = nlp(sentence)
    verb_lst = [(i, token.text) for i, token in enumerate(doc) if token.pos_ == "VERB"]
    possible = ["is", "was", "are", "were"]
    aux_lst = [(i, token.text) for i, token in enumerate(doc) if token.pos_ == "AUX"]
    # Valid = 0
    for verb_id, verb in verb_lst: # verb: (index, text)
        select_SVO = parse_sentence(id, select_SVO, doc, verb_id)
        # if subj != None and obj != None:
        #     Valid = 1
        #     tmp = [subj, verb, obj]
        #     select_SVO[id].append(tmp)
    for aux_id, aux in aux_lst:
        select_SVO = parse_sentence(id, select_SVO, doc, aux_id)
        # if subj != None and obj != None:
        #     Valid = 1
        #     tmp = [subj, verb, obj]
        #     select_SVO[id].append(tmp)
    # determine label
    # if Valid:
    # print(select_SVO)
    for tmps in select_SVO:
        for tmp in tmps:
            subj, verb, obj = tmp
            if subj in S[id] and verb in V[id] and obj in O[id]:
                y_pred[id] = 1
                s_pred[id] = subj
                v_pred[id] = verb
                o_pred[id] = obj
                break
    # if Valid == 0:
    #     y_pred.append(0)
    # else:
        # y_pred.append(0)
print(y_pred)
print(select_SVO[17])

if Inference:
    index = [i for i in range(len(y_pred))]
    dict = {"index": index, "T/F": y_pred} 
    output_df = pd.DataFrame(dict)
    output_df.to_csv("prediciton_3.csv", index= False)
else:
    dict = { "T/F": y_pred, "S": s_pred, "V": v_pred, "O": o_pred} # "index": index, 
    output_df = pd.DataFrame(dict)
    output_df.to_csv("test.csv", header=False, index= False)


[0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 

In [8]:
check = 18
print(sentences[check-1])

In broad terms , the statement released by Mar 'ie addressed these concerns .


In [9]:
spacy.displacy.render(nlp(sentences[check-1]), style='dep')

In [10]:
for token in nlp(sentences[check-1]):
    # print(token.text, list(token.subtree))
    print(token.text, token.pos_, token.dep_, f"< {spacy.explain(token.dep_)} >", f"[{token.head}]")

In ADP prep < prepositional modifier > [addressed]
broad ADJ amod < adjectival modifier > [terms]
terms NOUN pobj < object of preposition > [In]
, PUNCT punct < punctuation > [addressed]
the DET det < determiner > [statement]
statement NOUN nsubj < nominal subject > [addressed]
released VERB acl < clausal modifier of noun (adjectival clause) > [statement]
by ADP agent < agent > [released]
Mar PROPN compound < compound > [']
' PROPN pobj < object of preposition > [by]
ie PROPN pobj < object of preposition > [by]
addressed VERB ROOT < root > [addressed]
these DET det < determiner > [concerns]
concerns NOUN dobj < direct object > [addressed]
. PUNCT punct < punctuation > [addressed]


In [11]:
for token in nlp(sentences[check-1]):
    print(token.text, list(token.subtree))

In [In, broad, terms]
broad [broad]
terms [broad, terms]
, [,]
the [the]
statement [the, statement, released, by, Mar, ', ie]
released [released, by, Mar, ', ie]
by [by, Mar, ', ie]
Mar [Mar]
' [Mar, ']
ie [ie]
addressed [In, broad, terms, ,, the, statement, released, by, Mar, ', ie, addressed, these, concerns, .]
these [these]
concerns [these, concerns]
. [.]


In [12]:
a = [[(None, None, None)]]*3
print(a)
b = (1,2,3)
print(b)

[[(None, None, None)], [(None, None, None)], [(None, None, None)]]
(1, 2, 3)


In [13]:
aa = a[2]
print(aa)
aa.append(b)
a


[(None, None, None)]


[[(None, None, None), (1, 2, 3)],
 [(None, None, None), (1, 2, 3)],
 [(None, None, None), (1, 2, 3)]]